In [36]:
import pandas as pd


weather_df = pd.read_csv('weatherAUS.zip',
                            )
#null n/a N/A

weather_df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [37]:
weather_df.RainTomorrow.value_counts()

No     110316
Yes     31877
Name: RainTomorrow, dtype: int64

In [38]:
weather_df.RainTomorrow = weather_df.RainTomorrow.apply(lambda x: 1 if x == "Yes" else 0)

In [39]:
weather_df.shape

(145460, 23)

In [40]:
import torch


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [41]:
DEVICE

device(type='cuda', index=0)

In [42]:
weather_df.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow       int64
dtype: object

In [43]:
weather_df.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow         0
dtype: int64

In [44]:
weather_df.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow         0
dtype: int64

In [45]:
from sklearn.preprocessing import LabelEncoder
# df.to_dummies

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

# Convert the 'Date' column to datetime data type
weather_df['Date'] = pd.to_datetime(weather_df['Date'])

# Apply label encoding to categorical columns
# a, b, c : is_a; is_b
label_encoders = {}
categorical_columns = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']

for column in categorical_columns:
    le = LabelEncoder()
    weather_df[column] = le.fit_transform(weather_df[column].astype(str))
    label_encoders[column] = le

# Handle missing values - for simplicity, we'll fill missing values with the mean of each column
weather_df  = weather_df.fillna(weather_df.mean())

# Normalize numerical columns
numerical_columns = weather_df.select_dtypes(['float64']).columns
weather_df[numerical_columns] = (weather_df[numerical_columns] - weather_df[numerical_columns].mean()) / weather_df[numerical_columns].std()

weather_df.head()

<ipython-input-45-d915ce608fc7>:20: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  weather_df  = weather_df.fillna(weather_df.mean())


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,2,0.189446,-0.045336,-0.210071,-2.809321e-16,-6.508028e-16,13,0.302233,13,...,0.112394,-1.442960,-1.478783,-1.223878,1.568031e+00,-8.487318e-16,-0.014053,0.017023,0,0
1,2008-12-02,2,-0.753098,0.265042,-0.281649,-2.809321e-16,-6.508028e-16,14,0.302233,6,...,-1.319604,-1.296413,-1.047778,-1.118835,-3.920271e-16,-8.487318e-16,0.032464,0.381984,0,0
2,2008-12-03,2,0.110901,0.349691,-0.281649,-2.809321e-16,-6.508028e-16,15,0.454692,13,...,-1.637826,-1.052167,-1.493645,-0.983781,-3.920271e-16,-1.199228e+00,0.621683,0.221401,0,0
3,2008-12-04,2,-0.470335,0.674177,-0.281649,-2.809321e-16,-6.508028e-16,4,-1.222360,9,...,-1.266567,-1.736055,-0.007422,-0.368532,-3.920271e-16,-8.487318e-16,0.172016,0.703149,0,0
4,2008-12-05,2,0.833518,1.280826,-0.162353,-2.809321e-16,-6.508028e-16,13,0.073544,1,...,0.695801,-0.905620,-1.018054,-1.388945,1.126648e+00,1.667533e+00,0.125499,1.170298,0,0


In [46]:
weather_df.isna().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [47]:
weather_df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [48]:
import time
from tqdm import tqdm



for i in tqdm(range(10000)):
    time.sleep(0.005)
    pass

  5%|▍         | 480/10000 [00:02<00:49, 192.76it/s]


KeyboardInterrupt: ignored

In [49]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from tqdm import tqdm # to see the progress

# 1. Prepare the data
features = ['Rainfall', 'Humidity3pm', 'Pressure3pm', 'Temp3pm','Cloud3pm']
X = weather_df[features].values
y = weather_df['RainTomorrow'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)



In [50]:
y_test_tensor

tensor([[1.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])

In [51]:
import torch.nn as nn

# Parametrlər
input_size = 5  # 5 azad göstərici - independent feature - X features
hidden_size = 10  # ixtiyari gizli laydakı neyron sayı
output_size = 1  # çıxış. ya yağır, ya yağmır

# neyron şəbəkəsini `nn.Sequential` ilə edirik
model = nn.Sequential(
    nn.Linear(input_size, hidden_size),  # İlk tam bağlı lay
    nn.ReLU(),  # Aktivasiya funksiyası lambda x: max(x, 0)
    nn.Linear(hidden_size, hidden_size),  # Gizli lay
    nn.ReLU(),  #
    nn.Linear(hidden_size, output_size),  # Çıxış layı
    nn.Sigmoid(),  # Sigmoid aktivasiya funksiyası)
)

# itki funksiyası
loss_function = nn.BCELoss() # binary cross entropy (default)
# optimallaşdırıcı - nə idi bu?? izah edin
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# dövrələrin sayı
epochs = 10

# Modelin təlimi
for epoch in tqdm(range(epochs)):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

# 4. proqnoz və dəyərləndirmə
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_classes = (predictions > 0.5).int().numpy()

print("Dəqiqlik:", accuracy_score(y_test, predicted_classes))
print(classification_report(y_test, predicted_classes))


100%|██████████| 10/10 [00:31<00:00,  3.17s/it]

Dəqiqlik: 0.8365186305513543
              precision    recall  f1-score   support

           0       0.85      0.96      0.90     22672
           1       0.73      0.41      0.53      6420

    accuracy                           0.84     29092
   macro avg       0.79      0.68      0.71     29092
weighted avg       0.82      0.84      0.82     29092



In [52]:
# Parametrlər
input_size = 5  # 4 azad göstərici - independent feature - X features
hidden_size = 50  # ixtiyari gizli laydakı neyron sayı
output_size = 1  # çıxış. ya yağır, ya yağmır

# neyron şəbəkəsini `nn.Sequential` ilə təyin edirik
model = nn.Sequential(
    nn.Linear(input_size, hidden_size),  # İlk tam bağlı lay
    nn.ReLU(),  # Aktivasiya funksiyası
    nn.Linear(hidden_size, hidden_size),  # Gizli lay
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),  # Gizli lay
    nn.ReLU(),
    nn.Linear(hidden_size, output_size),  # Çıxış layı
    nn.Sigmoid()  # Sigmoid aktivasiya funksiyası
)


# itki funksiyası
loss_function = nn.BCELoss() # binary cross entropy (default)
# optimallaşdırıcı - nə idi bu?? izah edin
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# dövrələrin sayı
epochs = 50

# Modelin təlimi
for epoch in tqdm(range(epochs)):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

# 4. proqnoz və dəyərləndirmə
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_classes = (predictions > 0.5).int().numpy()

print("Dəqiqlik:", accuracy_score(y_test, predicted_classes))
print(classification_report(y_test, predicted_classes))


100%|██████████| 50/50 [03:07<00:00,  3.75s/it]

Dəqiqlik: 0.8371029836381135
              precision    recall  f1-score   support

           0       0.86      0.95      0.90     22672
           1       0.71      0.44      0.55      6420

    accuracy                           0.84     29092
   macro avg       0.78      0.70      0.72     29092
weighted avg       0.82      0.84      0.82     29092

